In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
import pickle

In [12]:
df = pd.read_csv("../data/city_day.csv")
df.head()

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Ahmedabad,2015-01-02,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2,Ahmedabad,2015-01-03,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
3,Ahmedabad,2015-01-04,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
4,Ahmedabad,2015-01-05,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN


In [13]:
# aqi, co, no, no2, o3, so2, pm2_5, pm10, nh3

In [14]:
# df_raw.AQI_Bucket.value_counts()

In [15]:
df = df.drop(["City","Date", "NOx", "Benzene", "Toluene", "Xylene", "AQI_Bucket"], axis=1)
df.head()

,PM2.5,PM10,NO,NO2,NH3,CO,SO2,O3,AQI
0,NaN,NaN,0.92,18.22,NaN,0.92,27.64,133.36,NaN
1,NaN,NaN,0.97,15.69,NaN,0.97,24.55,34.06,NaN
2,NaN,NaN,17.40,19.30,NaN,17.40,29.07,30.70,NaN
3,NaN,NaN,1.70,18.48,NaN,1.70,18.59,36.08,NaN
4,NaN,NaN,22.10,21.42,NaN,22.10,39.33,39.31,NaN


In [16]:
df.shape

(29531, 9)

In [17]:
knn_imputer = KNNImputer(n_neighbors = 3)
df = pd.DataFrame(knn_imputer.fit_transform(df), columns = df.columns)

In [18]:
df.head()

,PM2.5,PM10,NO,NO2,NH3,CO,SO2,O3,AQI
0,32.543333,81.630000,0.92,18.22,11.090000,0.92,27.64,133.36,97.000000
1,51.840000,269.710000,0.97,15.69,10.820000,0.97,24.55,34.06,118.333333
2,116.706667,183.263333,17.40,19.30,8.926667,17.40,29.07,30.70,396.666667
3,52.386667,89.370000,1.70,18.48,14.440000,1.70,18.59,36.08,88.333333
4,79.573333,182.506667,22.10,21.42,9.066667,22.10,39.33,39.31,435.000000


In [19]:
# from sklearn import linear_model
# from sklearn import ensemble
# from xgboost import XGBRegressor
# MODELS = {
#     "randomforest": ensemble.RandomForestRegressor(n_jobs=-1,verbose=0),
#     "lin_reg": linear_model.LinearRegression(),
#     "xgb": XGBRegressor(),

# }
# FOLD_MAPPING = {
#     0: [1, 2, 3, 4],
#     1: [0, 2, 3, 4],
#     2: [0, 1, 3, 4],
#     3: [0, 1, 2, 4],
#     4: [0, 1, 2, 3]
# }


In [20]:

# def train(fold, df, model, save_model=False):
#     """
#     Function to train the model fold-wise
#     :param fold: number of folds to run
#     :param model: model to train (key from model_dispatcher)
#     :param df: training data (pandas DataFrame)
#     :param save_model: to save model or not (True to save)
#     """

#     train_df = df[df.kfold.isin(FOLD_MAPPING.get(fold))]
#     valid_df = df[df.kfold == fold]

#     ytrain = train_df.AQI.values
#     yvalid = valid_df.AQI.values

#     train_df = train_df.drop(["AQI", "kfold"], axis=1)
#     valid_df = valid_df.drop(["AQI", "kfold"], axis=1)

#     # just to make sure all have same columns
#     valid_df = valid_df[train_df.columns]


#     reg = MODELS[model]
#     reg.fit(train_df, ytrain)
#     preds = reg.predict(valid_df)
#     score = metrics.r2_score(yvalid, preds)
#     print(f"fold = {fold}, score: {score}")
    
#     return score
    
    
# total = 0    
# for fold in range(5):

#     score = train(fold, df_fold, model="randomforest")
#     total += score
# print(f"Mean score: {total/5}")

In [21]:
X = df[df.columns[:-1]]
y = df.AQI

In [22]:

from sklearn.model_selection import train_test_split
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [23]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_jobs=-1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
score = metrics.r2_score(y_test, rf_pred)
print(score)

0.9005550425994858


In [25]:
import joblib
pickle.dump(rf, open("../aqi_rf.pkl", 'wb'))

In [16]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression(n_jobs=-1)
lr.fit(X, y)
import joblib
joblib.dump(rf, "aqi_lr.pkl")

['aqi_lr.pkl']

In [28]:
rf = pickle.load(open('../aqi_rf.pkl', 'rb'))
# PM2.5	PM10	NO	NO2	NH3	CO	SO2	O3
inp = np.array([[32.543333,	240.630000,	0.92,	40.22,	11.090000,	0.98,	27.64,	133.36	]])
print(rf.predict(inp))

[200.66333333]
